# Avaliação Literaria

Atualmente o nosso cliente realiza as analises manualmente e com cinco analistas. E para uxiliar os analistas iremos utilizar técnicas de NLP para extrair insights das colunas textuais da base de dados, focando em entender o desempenho dos autores, gêneros e identificar usuários com opiniões relevantes.

- Opiniões de sites sobre os livros mais lidos e mais vendidos;
    - [Os 20 livros mais lidos de todos os tempos](https://livrariafalada.com.br/os-20-livros-mais-lidos-de-todos-os-tempos/)
    - [Os 60 livros mais vendidos de todos os tempos](https://escritaselvagem.com.br/carreira-literaria/livros-mais-vendidos/)
- Sites de avaliações;
    - [FantasyBookReview](https://www.fantasybookreview.co.uk/)
    - [LibraryThing](https://www.librarything.com/)
    - [LoveReading](https://www.lovereading.co.uk/)

Por isso, faremos uma análise detalhada de como os autores estão sendo avaliados.

Esse notebook faz parte de um case. E os datasets utilizados foram [books_data (en)](https://drive.google.com/file/d/1-gmeMK-PmZe6rYAQPvXKdUXfD01uIbsd/view) e [books_rating (en)](https://drive.google.com/file/d/1v-OK4cJqSF4dcheIQlK1-W6JPbOpV44j/view).

Estrutura do notebook:

- Configuração do ambiente de desenvolvimento:
    - Instalações das Bibliotecas e configuração do ambiente
- Analise exploratoria dos dados:
    - Visualizações dos Dataframe
- Categorias dos livros:
    - Visualizando e categorizando os dados sem essa informação
- Avaliações dos livros:
    - Visualizando as avaliações dos livros
- Analises finais:
    - Retiramos mais alguns insight dos dados
- Conclusão

As colunas são:

 - books_data:
     - Title - O título do livro
     - description - Resumo do livro
     - authors - O autor do livro
     - image - Link com a imagem da capa do livro
     - previewLink - Link do livro no google books
     - publisher - Editora que publicou o livro
     - publishedDate - Data da publicação
     - infoLink - Link do livro no google books
     - categories - Categoria do livro
     - ratingsCount - Quantidade de resenhas do livro

 - Books_rating:
     - Id - Número de indentificação da avaliação
     - Title - O título do livro
     - Price - Valor do livro
     - User_id - Identificação do usuario que fez a avaliação
     - profileName - Nome do usuario que fez a avaliação
     - score - Avaliação do livro
     - time - Tempo de leitura do usuario
     - summary - Titulo da avaliação
     - text - Comentario do usuario

## Configuração do ambiente de desenvolvimento:

Para iniciar o desenvolvimento, é fundamental identificar e instalar as bibliotecas necessárias. Em seguida, essas bibliotecas devem ser importadas para o projeto.

In [1]:
# Instalação das bibliotecas necessárias para o projeto:

# %pip install pandas -q
# %pip install torch -q
# %pip install bertopic
# # %pip install umap-learn -q
# # %pip install hdbscan -q
# # %pip install sentence-transformers -q

In [2]:
# Bibliotecas utilizadas:

import os
import torch  # type: ignore
import pandas as pd  # type: ignore
import numpy as np  # type: ignore
import matplotlib.pyplot as plt  # type: ignore

from bertopic import BERTopic  # type: ignore

/Users/thiagogusson/Downloads/outros/Desafio/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/thiagogusson/Downloads/outros/Desafio/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Configurando a memória dinâmica da GPU:

memoria_dinamica = os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
print(f"Dynamic memory: {memoria_dinamica}")

Dynamic memory: true


In [4]:
# Configurando o Pandas:

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [5]:
# Verificando as GPUs com Torch:

mps_available = torch.backends.mps.is_available()
mps_built = torch.backends.mps.is_built()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print(f"Backend MPS está disponível? {mps_available}")
print(f"PyTorch foi construído com suporte a MPS? {mps_built}")
print(f"Qual device: {device}")

Backend MPS está disponível? True
PyTorch foi construído com suporte a MPS? True
Qual device: mps


## Análise dos dados

In [6]:
# Carregando o Dataframe:

df_books_data = pd.read_csv("./books_data.csv")
df_Books_rating = pd.read_csv("./Books_rating.csv")

In [7]:
# Visualizando o dataframe:

df_books_data.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [8]:
# Visualizando o dataframe:

df_Books_rating.head()

,Id,Title,Price,User_id,profileName,score,time,summary,text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.00,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,5.00,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,5.00,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.00,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.00,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [9]:
# Visualizando as informações do dataframe:

df_books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [10]:
# Visualizando as informações do dataframe:

df_Books_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Id           object 
 1   Title        object 
 2   Price        float64
 3   User_id      object 
 4   profileName  object 
 5   score        float64
 6   time         int64  
 7   summary      object 
 8   text         object 
dtypes: float64(2), int64(1), object(6)
memory usage: 206.0+ MB


In [11]:
# Visualizando as quantidades de linhas e colunas do dataframe:

df_books_data.shape, df_Books_rating.shape

((212404, 10), (3000000, 9))

A fim de otimizar a análise, as colunas image, previewLink, infoLink, publisher e ratingsCount podem ser removidas

In [12]:
df_books_data = df_books_data.drop(
    columns=[
        "image",
        "previewLink",
        "infoLink",
        "ratingsCount",
        "publisher"
    ]
)

Durante a análise, identificamos inconsistências na escrita dos nomes do autor e da obra. Padronizaremos o nome do autor para J.R.R. Tolkien e o título da obra para 'The Hobbit There and Back Again'.

In [13]:
mapeamento_nomes = {"['John Ronald Reuel Tolkien']": "['J. R. R. Tolkien']"}
df_books_data['authors'] = df_books_data['authors'].replace(mapeamento_nomes)

In [14]:
mapeamento_nomes = {"The Hobbit or There and Back Again": "The Hobbit There and Back Again",
                    "The Hobbitt, or there and back again; illustrated by the author.":"The Hobbit There and Back Again"}
df_books_data['Title'] = df_books_data['Title'].replace(mapeamento_nomes)

Vamos extrair apenas o ano de publicação, desconsiderando dia e mês.

In [15]:
# Função para converter datas para datetime e extrair o ano

def extract_year(date_str):
    for fmt in ('%Y-%m-%d', '%Y-%m', '%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt).year
        except ValueError:
            continue
    return None

# Aplicar a função para extrair o ano
df_books_data['publishedDate'] = df_books_data['publishedDate'].apply(extract_year)
df_books_data["publishedDate"] = df_books_data["publishedDate"].fillna(9999.00)
df_books_data['publishedDate'] = df_books_data['publishedDate'].astype(int).astype(str)

Remover possiveis duplicadas nos dados

In [16]:
df_books_data.drop_duplicates(inplace=True)

Para garantir a integridade dos dados, substituiremos os valores ausentes na coluna 'description' por um texto vázio, indicando a falta de informação.

In [17]:
df_books_data["description"] = df_books_data["description"].fillna("")

In [18]:
df_books_data.head()

,Title,description,authors,publishedDate,categories
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels']
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography']
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion']
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction']
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,NaN


In [19]:
# Visualizando as informações do dataframe:

df_books_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Title          212403 non-null  object
 1   description    212404 non-null  object
 2   authors        180991 non-null  object
 3   publishedDate  212404 non-null  object
 4   categories     171205 non-null  object
dtypes: object(5)
memory usage: 8.1+ MB


Visualizando os dados, percebemos que há títulos sem autores cadastrados. Considerando que o objetivo da nossa análise é avaliar o desempenho dos autores, esses registros não são relevantes e podem ser removidos.

In [20]:
df_books_data_ = df_books_data.dropna(subset=["authors"])

In [21]:
# Visualizando as informações do dataframe:

df_books_data_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180991 entries, 0 to 212403
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Title          180990 non-null  object
 1   description    180991 non-null  object
 2   authors        180991 non-null  object
 3   publishedDate  180991 non-null  object
 4   categories     165744 non-null  object
dtypes: object(5)
memory usage: 8.3+ MB


## Categorias dos livros

Dado que um dos objetivos do projeto é analisar a performance por gênero e constatamos a ausência de categorias em alguns livros, aplicaremos o modelo de tópicos [BERTopic](https://maartengr.github.io/BERTopic/) para inferir a categoria mais provável para cada um deles.

In [22]:
# Criando os Dataframes com e sem as categorias:

df_books_data_sem_categories = df_books_data_[df_books_data_["categories"].isna()]
df_books_data_com_categories = df_books_data_.dropna(subset=["categories"])

In [23]:
df_books_data_sem_categories.head()

,Title,description,authors,publishedDate,categories
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,NaN
6,The Overbury affair (Avon),,['Miriam Allen De Ford'],1960,NaN
7,A Walk in the Woods: a Play in Two Acts,,['Lee Blessing'],1988,NaN
27,"Thomas Jefferson, (American statesmen)",,['John T. Morse'],1888,NaN
29,Dealing With Disappointment: Helping Kids Cope...,Why Dragons So Angry? Despite our best efforts...,['John Adrian'],2020,NaN


In [24]:
df_books_data_com_categories.head()

,Title,description,authors,publishedDate,categories
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels']
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography']
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion']
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction']
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],1996,['Religion']


In [25]:
# Para utilizar o BERTopic, precisamos transformar os dataframes em listas:

documents_com_categories = df_books_data_com_categories.apply(
    lambda row: f"Title: {row['Title']} - Description: {row['description']}", axis=1
).tolist()

In [26]:
# Extraindo as categorias:

target_names = df_books_data_com_categories["categories"].unique().tolist()

In [27]:
# Mapeiando as categorias para índices numéricos:

category_to_index = {category: index for index, category in enumerate(target_names)}
target = df_books_data_com_categories["categories"].map(category_to_index).to_numpy()

In [28]:
print(f"Textos: {len(documents_com_categories)}")  # noqa: F821
print(f"Categorias: {len(target)}")

Textos: 165744
Categorias: 165744


In [29]:
if len(documents_com_categories) == len(target):  # noqa: F821
    topic_model = BERTopic()
    topics, probs = topic_model.fit_transform(documents_com_categories, y=target)  # noqa: F821
else:
    print("Os comprimentos dos documentos e dos rótulos não são iguais.")

In [30]:
# Salvanado o modelo treinado:

topic_model.save("./modelo/bertopic_model")

In [31]:
# Carregando o modelo treinado:

topic_model_loaded = BERTopic.load("./modelo/bertopic_model")

In [32]:
# Criando a lista sem categorias:

documents_sem_categories = df_books_data_sem_categories.apply(
    lambda row: f"Title: {row['Title']} - Description: {row['description']}", axis=1
).tolist()

In [33]:
# Prevendo as categorias dos novos textos:

new_topics, _ = topic_model_loaded.transform(documents_sem_categories)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [34]:
# Mapeando os índices numéricos para categorias:

index_to_category = {index: category for category, index in category_to_index.items()}

In [35]:
predicted_categories = [index_to_category[topic] if topic != -1 else "['others']" for topic in new_topics]

In [36]:
df_books_data_sem_categories["categories"] = predicted_categories

In [37]:
df_books_data_sem_categories.head()

,Title,description,authors,publishedDate,categories
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization']
6,The Overbury affair (Avon),,['Miriam Allen De Ford'],1960,['others']
7,A Walk in the Woods: a Play in Two Acts,,['Lee Blessing'],1988,"['Poets, English']"
27,"Thomas Jefferson, (American statesmen)",,['John T. Morse'],1888,"['Comic books, strips, etc']"
29,Dealing With Disappointment: Helping Kids Cope...,Why Dragons So Angry? Despite our best efforts...,['John Adrian'],2020,['others']


Identificamos que algumas categorias não possuem nomes ou descrições suficientemente detalhadas para serem classificadas automaticamente. Para garantir a precisão da classificação, solicitaremos aos analistas que realizem uma pesquisa mais aprofundada sobre essas categorias, coletando informações adicionais que permitam a criação de um modelo de classificação mais robusto no futuro.

In [38]:
df_books_data_1 = pd.concat([df_books_data_com_categories, df_books_data_sem_categories], axis=0).sort_index()

In [39]:
df_books_data_1.head()

,Title,description,authors,publishedDate,categories
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels']
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography']
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion']
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction']
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization']


## Avaliações dos livros

Quando de fala em avaliação de produtos a escala de avaliação da pesquisa é um dos tipos de questionários mais utilizados em pesquisas online e offline. Sua popularidade se deve ao fato de permitir a aplicação de medidas quantitativas que abrangem sentimentos mais abstratos e subjetivos.

Satisfação, experiência, percepção e sentimentos são difíceis de quantificar, mas uma escala de avaliação bem construída é útil para medir desempenho ou eficácia e possui aplicação em pesquisas de opinião.

Além disso, é útil para obter informações detalhadas sobre um tema específico, comparar e analisar dados, bem como medir elementos importantes de produtos ou serviços.

Por esse motivo devemos olhar essas escalas de avaliações.

saiba mais sobre escala de avaliação da pesquisa: [aqui](https://www.questionpro.com/blog/pt-br/escala-de-avaliacao-da-pesquisa/#:~:text=Uma%20Escala%20de%20Avalia%C3%A7%C3%A3o%20Num%C3%A9rica%2C%20como%20o%20nome,indicando%20a%20presen%C3%A7a%20de%20um%20sintoma%20muito%20severo.)

In [40]:
df_Books_rating.head()

,Id,Title,Price,User_id,profileName,score,time,summary,text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.00,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,5.00,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,5.00,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.00,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.00,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [41]:
df_Books_rating.describe()

,Price,score,time
count,481171.00,3000000.00,3000000.00
mean,21.76,4.22,1132306772.63
std,26.21,1.20,149320187.53
min,1.00,1.00,-1.00
25%,10.78,4.00,999907200.00
50%,14.93,5.00,1128297600.00
75%,23.95,5.00,1269129600.00
max,995.00,5.00,1362355200.00


In [42]:
# Visualizando a pontuação:

df_Books_rating["score"].value_counts()

score
5.00    1807343
4.00     585616
3.00     254295
1.00     201688
2.00     151058
Name: count, dtype: int64

In [43]:
# Visualizando os títulos:

df_Books_rating["Title"].value_counts().head()

Title
The Hobbit             22023
Pride and Prejudice    20371
Atlas Shrugged         12513
Wuthering Heights      10780
The Giver               7644
Name: count, dtype: int64

In [44]:
# Visualizando a pontuação:

df_Books_rating["Price"].value_counts().head()

Price
7.99     14729
14.95     8465
19.95     6341
24.95     6249
29.95     5612
Name: count, dtype: int64

Para melhorar a compreensão dos dados, seguiremos um processo que envolve algumas etapas.

In [45]:
# Trazendo as informações do autor e a categorias para as avaliações:

df_Books_rating_with_authors = pd.merge(
    df_books_data_1, df_Books_rating, on="Title", how="left"
)

In [46]:
df_Books_rating_with_authors.head()

,Title,description,authors,publishedDate,categories,Id,Price,User_id,profileName,score,time,summary,text
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.00,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A30TK6U7DNS82R,Kevin Killian,5.00,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A3UH4UZ4RSVO82,John Granger,5.00,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.00,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.00,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [47]:
# Contagem das avaliações de cada livro, autor e categoria:

df_Books_rating_with_authors["QtdDeAvaliacoesLivro"] = df_Books_rating_with_authors.groupby(
    "Title"
)["score"].transform("count")

df_Books_rating_with_authors["QtdDeAvaliacoesAutor"] = df_Books_rating_with_authors.groupby(
    "authors"
)["score"].transform("count")

df_Books_rating_with_authors["QtdDeAvaliacoesCateg"] = df_Books_rating_with_authors.groupby(
    "categories"
)["score"].transform("count")

In [48]:
df_Books_rating_with_authors.head()

,Title,description,authors,publishedDate,categories,Id,Price,User_id,profileName,score,time,summary,text,QtdDeAvaliacoesLivro,QtdDeAvaliacoesAutor,QtdDeAvaliacoesCateg
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.00,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1.00,1,12068
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A30TK6U7DNS82R,Kevin Killian,5.00,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,9.00,10,107002
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A3UH4UZ4RSVO82,John Granger,5.00,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",9.00,10,107002
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.00,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",9.00,10,107002
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.00,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,9.00,10,107002


In [49]:
# Contagem das avaliações de cada livro, autor e categoria:


df_Books_rating_with_authors["MediaAvaliacoesLivro"] = df_Books_rating_with_authors.groupby(
    "Title"
)["score"].transform(np.mean)

df_Books_rating_with_authors["MediaAvaliacoesAutor"] = df_Books_rating_with_authors.groupby(
    "authors"
)["score"].transform(np.mean)

df_Books_rating_with_authors["MediaAvaliacoesCateg"] = df_Books_rating_with_authors.groupby(
    "categories"
)["score"].transform(np.mean)

In [50]:
# Visualizando o dataframe:

df_Books_rating_with_authors.head()

,Title,description,authors,publishedDate,categories,Id,Price,User_id,profileName,score,time,summary,text,QtdDeAvaliacoesLivro,QtdDeAvaliacoesAutor,QtdDeAvaliacoesCateg,MediaAvaliacoesLivro,MediaAvaliacoesAutor,MediaAvaliacoesCateg
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",4.00,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,1.00,1,12068,4.00,4.00,4.34
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A30TK6U7DNS82R,Kevin Killian,5.00,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,9.00,10,107002,4.56,4.60,4.26
2,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A3UH4UZ4RSVO82,John Granger,5.00,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",9.00,10,107002,4.56,4.60,4.26
3,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",4.00,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",9.00,10,107002,4.56,4.60,4.26
4,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",4.00,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,9.00,10,107002,4.56,4.60,4.26


Como nossa análise terá como foco livro, autor e categoria, dividiremos os dados em dataframes distintos, um para cada critério. Iniciaremos nossa análise pela categoria.

In [51]:
df_Books_rating_with_categories_group = df_Books_rating_with_authors[
    ["categories", "score", "QtdDeAvaliacoesCateg", "MediaAvaliacoesCateg"]
]
df_Books_rating_with_categories_group.drop_duplicates(inplace=True, subset="categories")

In [52]:
# Visualizando o dataframe:

df_Books_rating_with_categories_group.head()

,categories,score,QtdDeAvaliacoesCateg,MediaAvaliacoesCateg
0,['Comics & Graphic Novels'],4.00,12068,4.34
1,['Biography & Autobiography'],5.00,107002,4.26
10,['Religion'],5.00,97141,4.34
14,['Fiction'],1.00,824183,4.11
46,['Technology and civilization'],5.00,5,4.60


In [53]:
# Visualizando informações detalhadas do dataframe:

df_Books_rating_with_categories_group.describe()

,score,QtdDeAvaliacoesCateg,MediaAvaliacoesCateg
count,10423.00,10423.00,10423.00
mean,4.34,250.35,4.28
std,1.07,8648.39,0.77
min,1.00,1.00,1.00
25%,4.00,1.00,4.00
50%,5.00,4.00,4.43
75%,5.00,13.00,5.00
max,5.00,824183.00,5.00


In [54]:
df_Books_rating_with_Title_group = df_Books_rating_with_authors[
    ["Id", "Title", "description", "score", "QtdDeAvaliacoesLivro", "MediaAvaliacoesLivro"]
]
df_Books_rating_with_Title_group.drop_duplicates(inplace=True, subset="Title")

In [55]:
# Visualizando o dataframe:

df_Books_rating_with_Title_group.head()

,Id,Title,description,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro
0,1882931173,Its Only Art If Its Well Hung!,,4.00,1.00,4.00
1,0826414346,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,5.00,9.00,4.56
10,0829814000,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,5.00,4.00,5.00
14,0595344550,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,1.00,32.00,3.72
46,0253338352,"Nation Dance: Religion, Identity and Cultural ...",,5.00,1.00,5.00


In [56]:
# Visualizando informações detalhadas do dataframe:

df_Books_rating_with_Title_group.describe()

,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro
count,180989.00,180988.00,180988.00
mean,4.35,14.42,4.27
std,1.06,120.36,0.82
min,1.00,1.00,1.00
25%,4.00,1.00,4.00
50%,5.00,3.00,4.50
75%,5.00,8.00,5.00
max,5.00,22023.00,5.00


In [57]:
df_Books_rating_with_authors_group = df_Books_rating_with_authors[
    ["authors", "score", "QtdDeAvaliacoesAutor", "MediaAvaliacoesAutor"]
]
df_Books_rating_with_authors_group.drop_duplicates(inplace=True, subset="authors")

In [58]:
# Visualizando o dataframe:

df_Books_rating_with_authors_group.head()

,authors,score,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor
0,['Julie Strain'],4.00,1,4.00
1,['Philip Nel'],5.00,10,4.60
10,['David R. Ray'],5.00,4,5.00
14,['Veronica Haddon'],1.00,32,3.72
46,['Edward Long'],5.00,1,5.00


In [59]:
# Visualizando informações detalhadas do dataframe:

df_Books_rating_with_authors_group.describe()

,score,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor
count,127277.00,127277.00,127277.00
mean,4.35,20.50,4.28
std,1.06,261.17,0.80
min,1.00,1.00,1.00
25%,4.00,1.00,4.00
50%,5.00,3.00,4.50
75%,5.00,9.00,5.00
max,5.00,50257.00,5.00


Tendo em vista que nosso principal objetivo é avaliar o desempenho de autores e categorias, vamos enriquecer o dataframe que contém os dados dos títulos e autores com as informações obtidas nas avaliações dos leitores.

In [60]:
# Visualizando o dataframe:

df_books_data_1.head()

,Title,description,authors,publishedDate,categories
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels']
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography']
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion']
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction']
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization']


In [61]:
# Visualizando o dataframe:

df_Books_rating_with_Title_group.head()

,Id,Title,description,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro
0,1882931173,Its Only Art If Its Well Hung!,,4.00,1.00,4.00
1,0826414346,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,5.00,9.00,4.56
10,0829814000,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,5.00,4.00,5.00
14,0595344550,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,1.00,32.00,3.72
46,0253338352,"Nation Dance: Religion, Identity and Cultural ...",,5.00,1.00,5.00


In [63]:
# Incluindo as avaliações nas informações dos livros:

df_books_data_1 = pd.merge(
    df_books_data_1,
    df_Books_rating_with_Title_group,
    on=["Title", "description"],
    how="left",
)

In [64]:
# Visualizando o dataframe:

df_books_data_1.head()

,Title,description,authors,publishedDate,categories,Id,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,4.00,1.00,4.00
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,5.00,9.00,4.56
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion'],0829814000,5.00,4.00,5.00
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction'],0595344550,1.00,32.00,3.72
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization'],0253338352,5.00,1.00,5.00


In [65]:
# Visualizando o dataframe:

df_Books_rating_with_authors_group.head()

,authors,score,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor
0,['Julie Strain'],4.00,1,4.00
1,['Philip Nel'],5.00,10,4.60
10,['David R. Ray'],5.00,4,5.00
14,['Veronica Haddon'],1.00,32,3.72
46,['Edward Long'],5.00,1,5.00


In [66]:
# Incluindo as avaliações nas informações dos autores:

df_books_data_1 = pd.merge(
    df_books_data_1,
    df_Books_rating_with_authors_group,
    on=["authors", "score"],
    how="left",
)

In [67]:
# Visualizando o dataframe:

df_books_data_1.head()

,Title,description,authors,publishedDate,categories,Id,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,4.00,1.00,4.00,1.00,4.00
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,5.00,9.00,4.56,10.00,4.60
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion'],0829814000,5.00,4.00,5.00,4.00,5.00
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction'],0595344550,1.00,32.00,3.72,32.00,3.72
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization'],0253338352,5.00,1.00,5.00,1.00,5.00


In [68]:
# Visualizando o dataframe:

df_Books_rating_with_categories_group.head()

,categories,score,QtdDeAvaliacoesCateg,MediaAvaliacoesCateg
0,['Comics & Graphic Novels'],4.00,12068,4.34
1,['Biography & Autobiography'],5.00,107002,4.26
10,['Religion'],5.00,97141,4.34
14,['Fiction'],1.00,824183,4.11
46,['Technology and civilization'],5.00,5,4.60


In [69]:
# Incluindo as avaliações nas informações dos categorias:

df_books_data_1 = pd.merge(
    df_books_data_1,
    df_Books_rating_with_categories_group,
    on=["categories", "score"],
    how="left",
)

In [70]:
# Visualizando o dataframe:

df_books_data_1.head()

,Title,description,authors,publishedDate,categories,Id,score,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor,QtdDeAvaliacoesCateg,MediaAvaliacoesCateg
0,Its Only Art If Its Well Hung!,,['Julie Strain'],1996,['Comics & Graphic Novels'],1882931173,4.00,1.00,4.00,1.00,4.00,12068.00,4.34
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],2005,['Biography & Autobiography'],0826414346,5.00,9.00,4.56,10.00,4.60,107002.00,4.26
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],2000,['Religion'],0829814000,5.00,4.00,5.00,4.00,5.00,97141.00,4.34
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],2005,['Fiction'],0595344550,1.00,32.00,3.72,32.00,3.72,824183.00,4.11
4,"Nation Dance: Religion, Identity and Cultural ...",,['Edward Long'],2003,['Technology and civilization'],0253338352,5.00,1.00,5.00,1.00,5.00,5.00,4.60


In [71]:
# Reorganizando o dataframe:

df_books_data_1 = df_books_data_1[["Title",
                                   "description",
                                   "QtdDeAvaliacoesLivro",
                                   "MediaAvaliacoesLivro",
                                   "publishedDate",
                                   "Id",
                                   "score",
                                   "authors",
                                   "QtdDeAvaliacoesAutor",
                                   "MediaAvaliacoesAutor",
                                   "categories",
                                   "QtdDeAvaliacoesCateg",
                                   "MediaAvaliacoesCateg"]]

In [73]:
# Visualizando o dataframe:

df_books_data_1.head()

,Title,description,QtdDeAvaliacoesLivro,MediaAvaliacoesLivro,publishedDate,Id,score,authors,QtdDeAvaliacoesAutor,MediaAvaliacoesAutor,categories,QtdDeAvaliacoesCateg,MediaAvaliacoesCateg
0,Its Only Art If Its Well Hung!,,1.00,4.00,1996,1882931173,4.00,['Julie Strain'],1.00,4.00,['Comics & Graphic Novels'],12068.00,4.34
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,9.00,4.56,2005,0826414346,5.00,['Philip Nel'],10.00,4.60,['Biography & Autobiography'],107002.00,4.26
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,4.00,5.00,2000,0829814000,5.00,['David R. Ray'],4.00,5.00,['Religion'],97141.00,4.34
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,32.00,3.72,2005,0595344550,1.00,['Veronica Haddon'],32.00,3.72,['Fiction'],824183.00,4.11
4,"Nation Dance: Religion, Identity and Cultural ...",,1.00,5.00,2003,0253338352,5.00,['Edward Long'],1.00,5.00,['Technology and civilization'],5.00,4.60


## Analises finais

Vamos agrupar o dataframe por autor e título, utilizando a média da nota, caso haja mais que uma, e a quantidade de avaliações como critério de desempate.

In [74]:
# Calculando os 10 livros mais bem avaliados:

df_books_data_1.groupby(['Id', 'authors', 'Title']).agg({'MediaAvaliacoesLivro':'mean', 'QtdDeAvaliacoesLivro':'sum'}).sort_values(
    by=['QtdDeAvaliacoesLivro', 'MediaAvaliacoesLivro'], ascending=False).head(10)

,,,MediaAvaliacoesLivro,QtdDeAvaliacoesLivro
Id,authors,Title,,
B000NDSX6C,['J. R. R. Tolkien'],The Hobbit,4.66,22023.00
1844560333,['Jane Austen'],Pride and Prejudice,4.53,20371.00
B000ILIJE0,['J. R. R. Tolkien'],The Hobbit There and Back Again,4.66,13314.00
1593355548,['Emily Bronte'],Wuthering Heights,4.05,10780.00
B000P1QRII,['Lois Lowry'],The Giver,4.27,7644.00
158726398X,['Charles Dickens'],Great Expectations,4.09,7421.00
B000IEZE3G,['J. K. Rowling'],Harry Potter and The Sorcerer's Stone,4.74,6796.00
B000P3LVZA,['Aldous Huxley'],Brave New World,4.24,6312.00
B000HKRIJO,['C. S. Lewis'],Mere Christianity,4.47,6053.00


O autor mais bem representado é o J. R. R. Tolkien, com os 2 volumes da saga Hobbit presentes na lista. Os outros autores tem apenas um livro cada na lista, com isso podemos ver que a grande maioria das pessoas gostam de ler livros de autores conhecidos.

Agora vejamos como a avaliação média varia por autor, com a quantidade de avaliações servindo como critério de desempate.

In [75]:
df_books_data_1.groupby(['authors']).agg({'MediaAvaliacoesAutor':'mean', 'QtdDeAvaliacoesAutor':'sum', 'Id':'nunique'}).sort_values(
    by=['MediaAvaliacoesAutor', 'QtdDeAvaliacoesAutor'], ascending=False).head(20)

,MediaAvaliacoesAutor,QtdDeAvaliacoesAutor,Id
authors,,,
['Louis Slobodkin'],5.00,275.00,5
['Randall Jarrell'],5.00,222.00,3
['George Santayana'],5.00,200.00,10
['David Seabury'],5.00,156.00,3
['Spike Milligan'],5.00,120.00,4
['Michelle Cole'],5.00,113.00,1
['Jack Barnes'],5.00,110.00,5
['John Keble'],5.00,105.00,7
['Esther Averill'],5.00,104.00,4


Com essa nova configuração, J. R. R. Tolkien e Jane Austen mal entrou nos 20 primeiros colocados. Isso provavelmente pode ser relacionado a quantidade de livros publicados, uma vez que a maioria dos autores na lista tem menos de 10 publicações. 

Mais publicações significa um risco maior de receber avaliações baixas.

Vamos ver se podemos dizer algo sobre a popularidade dos gêneros. Podemos calcular a quantidade de livros em cada gênero, exibindo os 20 gêneros com mais livros.

In [76]:
df_Books_rating_with_authors['categories'].value_counts().head(20)

categories
['Fiction']                      824183
['Juvenile Fiction']             204645
['Biography & Autobiography']    107002
['others']                       101267
['Religion']                      97141
['History']                       90082
['Business & Economics']          65533
['Computers']                     42449
['Social Science']                34269
['Cooking']                       29617
['Family & Relationships']        29331
['Self-Help']                     29002
['Young Adult Fiction']           28712
['Juvenile Nonfiction']           28515
['Body, Mind & Spirit']           28293
['Health & Fitness']              25229
['Science']                       24809
['Political Science']             22853
['Sports & Recreation']           21880
['Philosophy']                    20316
Name: count, dtype: int64

In [77]:
contagem_categorias = df_Books_rating_with_authors['categories'].value_counts().reset_index()

contagem_categorias.columns = ['categories', 'contagem']

media_avaliacoes = df_Books_rating_with_authors.groupby('categories')['score'].mean().reset_index()
media_avaliacoes.columns = ['categories', 'media_avaliacoes']

resultados_por_categoria = pd.merge(contagem_categorias, media_avaliacoes, on='categories')

resultados_por_categoria = resultados_por_categoria.sort_values(by='contagem', ascending=False)
resultados_por_categoria.head(20)

,categories,contagem,media_avaliacoes
0,['Fiction'],824183,4.11
1,['Juvenile Fiction'],204645,4.45
2,['Biography & Autobiography'],107002,4.26
3,['others'],101267,4.25
4,['Religion'],97141,4.34
5,['History'],90082,4.23
6,['Business & Economics'],65533,4.12
7,['Computers'],42449,3.96
8,['Social Science'],34269,4.13
9,['Cooking'],29617,4.40


In [78]:
resultados_por_categoria.describe()

,contagem,media_avaliacoes
count,10423.00,10423.00
mean,250.35,4.28
std,8648.39,0.77
min,1.00,1.00
25%,1.00,4.00
50%,4.00,4.43
75%,13.00,5.00
max,824183.00,5.00


Os dados indicam que os gêneros Ficção, Ficção Juvenil e Biografia/Autobiografia são os mais populares, apresentando uma média de avaliação de 4,27, próxima à média geral. No entanto, a presença da categoria 'Outros' entre os cinco primeiros sugere que uma parcela significativa dos livros não possui um gênero definido, o que dificulta uma análise mais precisa dos gêneros mais bem avaliados.

Vamos observar os 20 livros mais lidos. Para isso, agruparemos nossa tabela por autor e titulo, e calcularemos o total de pessoas que leram pelas avaliações feitas, adicionaremos o ano "mínimo" da publicação (caso haja mais de um ano).

In [79]:
# Livros mais lidos:

df_books_data_1.groupby(['Id', 'authors', 'Title']).agg({'QtdDeAvaliacoesLivro':'sum', 'publishedDate':'min'}).sort_values(
    by=['QtdDeAvaliacoesLivro'], ascending=False).head(20)

,,,QtdDeAvaliacoesLivro,publishedDate
Id,authors,Title,,
B000NDSX6C,['J. R. R. Tolkien'],The Hobbit,22023.00,2012
1844560333,['Jane Austen'],Pride and Prejudice,20371.00,1995
B000ILIJE0,['J. R. R. Tolkien'],The Hobbit There and Back Again,13314.00,2013
1593355548,['Emily Bronte'],Wuthering Heights,10780.00,9999
B000P1QRII,['Lois Lowry'],The Giver,7644.00,2014
158726398X,['Charles Dickens'],Great Expectations,7421.00,2003
B000IEZE3G,['J. K. Rowling'],Harry Potter and The Sorcerer's Stone,6796.00,2014
B000P3LVZA,['Aldous Huxley'],Brave New World,6312.00,2008
B000HKRIJO,['C. S. Lewis'],Mere Christianity,6053.00,2009


Analisaremos agora os mesmos pontos, mas agrupando por autor. Vamos começar pelos autores mais lidos, e adicionarem o ano "mínimo", ou seja, o ano do livro mais antigo do autor, isso pode nos ajudar a entender as preferências dos usuários. Vamos adicionar também a quantidade de publicações de cada autor.

In [80]:
# Autores mais lidos:

df_books_data_1.groupby(['authors']).agg({'QtdDeAvaliacoesLivro':'sum', 'publishedDate':'min', 'Id':'nunique'}).sort_values(
    by=['QtdDeAvaliacoesLivro'], ascending=False).head(20)

,QtdDeAvaliacoesLivro,publishedDate,Id
authors,,,
['J. R. R. Tolkien'],50257.00,1965,24
['Jane Austen'],36532.00,1818,53
['Charles Dickens'],21270.00,1837,89
['John Steinbeck'],17027.00,1937,54
['Lewis Carroll'],12847.00,1869,52
['Kurt Vonnegut'],12124.00,1961,35
['Harper Lee'],12013.00,1970,5
['Emily Bronte'],11872.00,1988,6
['C. S. Lewis'],11824.00,1999,42


Os autores no top 20 tem seus livros mais antigos publicados antes de 1990. Podemos interpretar, que os leitores preferem autores mais antigos. Também vemos como parece não haver uma relação entre número de publicações e número de leitores.

In [81]:
df_books_data_1.groupby('authors').agg({'Id':'nunique'}).describe()

,Id
count,127277.00
mean,1.42
std,2.15
min,1.00
25%,1.00
50%,1.00
75%,1.00
max,236.00


Com base nas análises realizadas, constatamos que o DataFrame já contém um volume significativo de informações relevantes. Ao aprofundar a análise, combinando os dados presentes neste notebook, podemos obter insights mais detalhados sobre o desempenho dos autores e gêneros literários. Dessa forma, estaremos aptos a responder de forma mais precisa às questões de pesquisa que nortearam este estudo.

In [82]:
# Leitores que mais leram:

df_Books_rating_with_authors.groupby(['profileName']).agg({'categories':'nunique', 'Id':'nunique', 'score':'min'}).sort_values(
    by=['Id'], ascending=False).head(20)

,categories,Id,score
profileName,,,
Midwest Book Review,491,4785,4.00
Harriet Klausner,245,3123,3.00
A Customer,348,2809,1.00
"Shalom Freedman ""Shalom Freedman""",272,1673,1.00
"Blue Tyson ""- Research Finished""",218,1496,1.00
John,201,1276,1.00
"Donald Mitchell ""Jesus Loves You!""",159,1164,2.00
Avid Reader,133,1088,1.00
Charles Ashbacher,232,1048,1.00


A análise do DataFrame, incluindo as avaliações por autor, nos permite identificar potenciais entrevistados com base no número de livros lidos. No entanto, para construir um modelo preciso de perfil de leitor e realizar recomendações eficazes, é imprescindível coletar informações adicionais sobre os usuários. A simples contagem de livros e categorias lidas não é suficiente para caracterizar adequadamente um perfil de leitura.

Por exemplo, a segunda colocada, apesar de ter lido um número expressivo de livros e categorias, apresenta uma média de avaliação relativamente baixa. Essa discrepância indica que a quantidade de livros e categorias lidas não se correlaciona diretamente com a complexidade ou profundidade das preferências de leitura. Portanto, é necessário considerar outros fatores, como a variedade de gêneros, autores favoritos, temas preferidos e até mesmo o tempo dedicado à leitura.

## Conclusões

A análise realizada revelou que as preferências dos leitores são complexas e multifacetadas, influenciadas por diversos fatores como a data de publicação, o gênero e a reputação do autor. Embora a quantidade de publicações não seja o único fator determinante da popularidade, ela desempenha um papel importante.

Além disso, identificamos a necessidade de uma classificação mais precisa dos gêneros e de dados mais detalhados sobre os leitores para construir modelos de recomendação mais precisos. Os resultados desta pesquisa podem ser utilizados para melhorar a experiência do leitor, otimizar as estratégias de marketing e impulsionar o crescimento da indústria editorial.